# Matanbuchus Triage Notes
> Matanbuchus loader triage and yara rule

- toc: true 
- badges: true
- categories: [Matanbuchus,loader,yara,triage,dumpulator,emulation]

![](https://cdn.discordapp.com/attachments/885652495421030431/988186871660183612/a_horse.png)

## Overview

Matanbuchus is a malware downloader that has been observed as the final loading stage in multiple phishing campaigns. There are two components, a Matanbuchus loader intented to load the Matanbuchus downloader. Once the downloader is executed it makes an HTTP request to the C2 with some victim info, and downloads the final payload.


According to [DCSO CyTec](https://twitter.com/DCSO_CyTec)...
>Matanbuchus is the name given to a Malware-as-a-Service sold on Russian-speaking cybercriminal forums. Starting at a rental price of $2,500, the malware consists of an obfuscated two-stage loader which has been deployed in conjunction with Qakbot and Cobalt Strike payloads.


![](https://isc.sans.edu/diaryimages/images/2022-06-17-ISC-diary-image-00a.jpg)


### References 
- [Malspam pushes Matanbuchus malware, leads to Cobalt Strike
](https://isc.sans.edu/forums/diary/Malspam+pushes+Matanbuchus+malware+leads+to+Cobalt+Strike/28752/)
- [Not Packed (UnpacMe)](https://www.unpac.me/results/7166936d-dcd6-40ed-9d15-0151953e46ff#/)
- [f8cc2cf36e193774f13c9c5f23ab777496dcd7ca588f4f73b45a7a5ffa96145e](https://malshare.com/sample.php?action=detail&hash=f8cc2cf36e193774f13c9c5f23ab777496dcd7ca588f4f73b45a7a5ffa96145e)
- [Matanbuchus: Malware-as-a-Service with Demonic Intentions](https://unit42.paloaltonetworks.com/matanbuchus-malware-as-a-service/)
- [A deal with the devil: Analysis of a recent Matanbuchus sample](https://medium.com/@DCSO_CyTec/a-deal-with-the-devil-analysis-of-a-recent-matanbuchus-sample-3ce991951d6a)
- [MATANBUCHUS: Another Loader As A Service Malware](https://www.0ffset.net/reverse-engineering/matanbuchus-loader-analysis/)
- [List of known versions of Windows DLLs](https://winbindex.m417z.com/?file=ntdll.dll)
- [Free Yara Scan Service](https://riskmitigation.ch/yara-scan/)
- [MalwareBazaar Client](https://github.com/larsborn/MalwareBazaarClient)

## Analysis

- The DLL has an export called `HackCheck` that uses OutputDebugStringA to print `start dll HackCheck` 
- The sample uses API hashing with FNV1a hash algo to resolve API calls


### Yara Rule

This yara rule is very brittle and needs lots of testing/refining

```

rule matanbuchus {
    meta:
        description = "Identifies matanbuchus"
        
   strings:
       // recursive fnv1 hash
       // 69 C2 93 01 00 01                       imul    eax, edx, 1000193h
       // 50                                      push    eax
       // B9 01 00 00 00                          mov     ecx, 1
       // C1 E1 00                                shl     ecx, 0
       $x1 = { 69 c2 93 01 00 01 50 b9 01 00 00 00 c1 e1 00 }
         
      //  string decryption    
      //  0F BE 1C 01                             movsx   ebx, byte ptr [ecx+eax]
      //  33 DE                                   xor     ebx, esi
      //  6A 00                                   push    0
      //  6A 01                                   push    1
        $x2 = { 0f be 1c 01 33 de 6a 00 6a 01 }
     
       
   condition:
       all of ($x*)
}


```

#### Yara Rule Revised

```
rule matanbuchus {
    meta:
        description = "Identifies matanbuchus"
        
   strings:
       // fnv1 hash
       // 69 C2 93 01 00 01                       imul    eax, edx, 1000193h
       // 
       // 69 C0 93 01 00 01                       imul    eax, 1000193h
       $x1 = { 69 ?? 93 01 00 01 }
         
      //pe loader
      $x2 = { B8 4D 5A 00 00 }
      $x3 = { 81 38 50 45 00 00 }
     
      // murmur
      $x4 = { 95 E9 D1 5B }
     
      //InternetCloseHandle
      $x5 = { 66 E9 DD 4D }
     
      //InternetOpenUrlA
      $x6 = { BC 8B CF F4 }
     
      //InternetCheckConnectionA
      $x7 = { 3F 82 58 52 }
      
      //InternetReadFile
      $x8 = { 66 E9 DD 4D }
       
   condition:
       all of ($x*)
}
```



### String Decryption
#### Simple Decryption
Some strings are built as stack strings then copied into a buffer and returned from a function. The returned buffer is then decrypted directly using a simple XOR decryption routine where the first byte is the key.

In [1]:
import struct

data = b'jl8|tt8Py{s[p}{s'
key = struct.pack('<I',0x796C6B18)
data = key[1:] + data
out = []


for i in data:
    out.append(i ^ key[0])

bytes(out)




b'start dll HackCheck'

In [2]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

#### Complex Decryption
Other strings are also build from stack strings in a function and returned in a buffer, but these strings are decrypted with a second function call to a decryption function. To handle these more complex strings, we will use [dumpulator](https://github.com/mrexodia/dumpulator)








In [3]:
from dumpulator import Dumpulator
import struct
import re
import pefile

file_data = open('/tmp/mat.bin','rb').read()
pe = pefile.PE(data=file_data)







In [4]:
dp = Dumpulator("/tmp/mat.dmp", quiet=True)
fn_get_string = 0x708641C0
ss_start = 0x708631B2
ss_end = 0x708632AA 

dp.start(ss_start, end=ss_end)

ss = dp.read(dp.regs.ebp-0x50, 0x48)




ss = bytes(ss)[:0x3e]

buff = dp.allocate(0x3e)
dp.write(buff, ss)
fn_dec_str = 0x7086F3D0 

dp.call(fn_dec_str, [buff, 0x3e, 0x0, 0x7ebbfa3, 0x1010101])
dp.read(buff,0x3e)

Failed to read module data


bytearray(b'https://telemetrysystemcollection.com/m8YYdu/mCQ2U9/auth.aspx\x00')

### Dumpulator Notes
The DLL uses thread safe functions that require a call into `EnterCriticalSection` and `LeaveCriticalSection`. Because our dump was taken before the DLL was initialized the critical section object was not initialized and this causes Dumpulator to crash.

First we tried just calling the initialization function in the DLL to setup this object (also tried calling the wrapper functions for `InitializeCriticalSectionEx`) but these all led to crashes (we could implement some syscalls and try again but we are lazy!

Our solution is to just patch out the `EnterCriticalSection` and `LeaveCriticalSection` calls in the loaded `ntdll` using a simple return 0.
```
33 C0     xor eax,eax
C2 04 00  ret 4
```
#### Dumpulator Patching Out Functions

In [5]:
dp = Dumpulator("/tmp/mat.dmp", quiet=True)

ntdll_start = 0x77a10000


patch_bytes = b'\x33\xC0\xC2\x04\x00'

ptr_RtlLeaveCriticalSection = 0x77A5FFF0
ptr_RtlEnterCriticalSection = 0x77A5FDF0

get_str_fn = 0x70861000

#tohex(dp.read(ntdll_init_crit_sec,10))

dp.write(ptr_RtlLeaveCriticalSection, patch_bytes)
dp.write(ptr_RtlEnterCriticalSection, patch_bytes)



dp.call(get_str_fn)


Failed to read module data
unmapped read from 8[4], cip = 77a6693a
error: Invalid memory read (UC_ERR_READ_UNMAPPED), cip = 77a6693a


134217727

#### Dumpulator Implementing Syscalls to Load DLL
Our patching method sort of worked but there is other initializations stuff that was causing more crashes. We are going to try and just implement the syscalls that we need to actually emulate the full DLL load routine and see if that works better.

In [6]:
from dumpulator import Dumpulator, syscall
from dumpulator.native import *

@syscall
def ZwQueryVolumeInformationFile(dp: Dumpulator,
                                 FileHandle: HANDLE,
                                 IoStatusBlock: P(IO_STATUS_BLOCK),
                                 FsInformation: PVOID,
                                 Length: ULONG,
                                 FsInformationClass: FSINFOCLASS
                                 ):
    return STATUS_SUCCESS



dp = Dumpulator("/tmp/mat2.dmp", quiet=True)

dll_base_addr = 0x71950000

dp.start(dp.regs.eip, end=dp.read_ptr(dp.regs.esp))


get_str_fn = 0x71951000
dp.read_str(dp.call(get_str_fn))



Failed to read module data


'https://telemetrysystemcollection.com/m8YYdu/mCQ2U9/auth.aspx'

#### Finding Encrypted Stack Strings


In [7]:
egg = rb'(\xC6\x45..){4}'

egg = rb'\x55\x8b\xec\x83\xec\x08\x33\xc0\x88\x45\xff'

stack_string_offsets = []
for m in re.finditer(egg, file_data):
    fn_offset = m.start()
    fn_addr = pe.get_rva_from_offset(fn_offset) + dll_base_addr
    tmp_str = dp.read_str(dp.call(fn_addr))
    print(tmp_str)

    
    



https://telemetrysystemcollection.com/m8YYdu/mCQ2U9/auth.aspx
https://collectiontelemetrysystem.com/m8YYdu/mCQ2U9/auth.aspx
DllRegisterServer
http://collectiontelemetrysystem.com/m8YYdu/mCQ2U9/home.aspx
http://telemetrysystemcollection.com/m8YYdu/mCQ2U9/home.aspx
https://telemetrysystemcollection.com/m8YYdu/mCQ2U9/home.aspx


## Revising Our Config Extractor

Now that we have a base config extractor we need to test this across multiple samples/versions of the malware to make sure it is robust enough.

Our initial sample was `f8cc2cf36e193774f13c9c5f23ab777496dcd7ca588f4f73b45a7a5ffa96145e`.

We have collected the following samples to triage:
- `b9b399dbb5d901c16d97b7c30cc182736cd83a7c53313194a1798d61f9c7501e`
- `c41f7b7ec0909d5c21107ddebc2fe84dbc137f701b42943c1a5e69f5d50e05ab` 
- `b4e7710488c2b7aaa71688b8bd546410af07a215c2e835e8dfbe24887186bd4f` 
- `60f030597c75f9df0f7a494cb5432b600d41775cfe5cf13006c1448fa3a68d8d` 
- `58a673023bbc7f2726e3b7ac917a43d9306692dc87b638ee21b98705a3262ccd` 
- `4b87f95c4477fc66c58b8e16a74f9c47217913cb4a78dc69f27a364a099acd90` 
- `4eb85a5532b98cbc4a6db1697cf46b9e2b7e28e89d6bbfc137b36c0736cd80e2`

These samples don't work with our first version of the config extractor:
- `e58b9bbb7bcdf3e901453b7b9c9e514fed1e53565e3280353dccc77cde26a98e.bin`
- `b9b399dbb5d901c16d97b7c30cc182736cd83a7c53313194a1798d61f9c7501e.bin`




### New Config Extractor - Static String Decryption Notes

In [25]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

In [89]:
import re
import struct
import pefile

FILE_PATH = '/tmp/samples/e58b9bbb7bcdf3e901453b7b9c9e514fed1e53565e3280353dccc77cde26a98e.bin'
FILE_PATH = '/tmp/samples/orig.bin'
file_data = open(FILE_PATH,'rb').read()



def xor_decrypt(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i%len(key)])
    return bytes(out)


def is_ascii(data):
    return re.match(B"^[\s!-~]+\0*$", data) is not None


stack_strings = []

#string_egg = rb'(\xC6\x45..){2,}'

string_egg = rb'(?P<a>(?:\xC6\x85.{5}){0,})(?P<b>(?:\xC6\x45..){3,})'   

for m in re.finditer(string_egg, file_data):
    match_data = m.group(0)
    #stack_strings.append({"data":match_data.replace(b'\xC6\x45',b'')[1::2], "match":match_data})
    stack_strings.append({"data":m['a'][6::7] + m['b'][3::4], "match":match_data})

    


keys = []

key_byte_len_egg = rb'\x68(....)\x68(....)\x6a\x00\x6a(.)'

for m in re.finditer(key_byte_len_egg, file_data):
     keys.append( {'key':m.group(2) + m.group(1), 'length':ord(m.group(3))})
        
        
key_dw_len_egg = rb'\x68(....)\x68(....)\x6a\x00\x68(....)'

for m in re.finditer(key_dw_len_egg, file_data):
     keys.append( {'key':m.group(2) + m.group(1), 'length':struct.unpack('<I',m.group(3))[0]})      

        
for sj in stack_strings:
    s = sj.get('data')
    str_len = len(s)
    #print(f"\n\n{str_len}")
    flag_found = False
    for k in keys:
        if k.get('length') == str_len:
            #print(f"found key: {tohex(k.get('key'))}")
            out = xor_decrypt(s, k.get('key'))
            if is_ascii(out):
                print(out)
                flag_found = True
    if not flag_found:
        print(xor_decrypt(s[1:-1], bytes([s[0]])))



b'LoadLibraryA'
b'GetProcAddress'
b'kernel32.dll'
b'kernel32.dll'
b'VirtualAlloc'
b'kernel32.dll'
b'VirtualFree'
b'kernel32.dll'
b'IPHLPAPI.DLL\x00'
b'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 10.0; Win64; x64; Trident/8.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; Microsoft Outlook 16.0.5197; ms-office; MSOffice 16)\x00'
b'Netapi32.dll\x00'
b'Shlwapi.dll\x00'
b'Wkscli.dll\x00'
b'%PROCESSOR_LEVEL%'
b'_qlinet.ltn\x00'
b'Wininet.dll\x00'
b'loaddll32.exe\x00'
b'USER32.dll\x00'
b'Netapi32.dll\x00'
b'^stapi32>rll\x00'
b'https://telemetrysystemcollection.com/m8YYdu/mCQ2U9/auth.aspx\x00'
b'Shlwapi.dll\x00'
b'WS2_32.dll\x00'
b'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 10.0; Win64; x64; Trident/8.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; Microsoft Outlook 16.0.5197; ms-office; MSOffice 16)\x00'
b'Wininet.dll\x00'
b'5"HLPAPIR6LL\x00'
b'IPHLPAPI.DLL\x00'
b'USER32.dll\x00'
b'DllRegisterServer\x00'
b'Shell32.dll\

### Final Config Decryptor 

In [97]:
import re
import struct
import pefile


def xor_decrypt(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i%len(key)])
    return bytes(out)


def is_ascii(data):
    return re.match(B"^[\s!-~]+\0*$", data) is not None


def get_strings(file_path):
    file_data = open(file_path,'rb').read()

    stack_strings = []

    #string_egg = rb'(\xC6\x45..){2,}'

    string_egg = rb'(?P<a>(?:\xC6\x85.{5}){0,})(?P<b>(?:\xC6\x45..){3,})'   

    for m in re.finditer(string_egg, file_data):
        match_data = m.group(0)
        #stack_strings.append({"data":match_data.replace(b'\xC6\x45',b'')[1::2], "match":match_data})
        stack_strings.append({"data":m['a'][6::7] + m['b'][3::4], "match":match_data})

    keys = []

    key_byte_len_egg = rb'\x68(....)\x68(....)\x6a\x00\x6a(.)'

    for m in re.finditer(key_byte_len_egg, file_data):
         keys.append( {'key':m.group(2) + m.group(1), 'length':ord(m.group(3))})


    key_dw_len_egg = rb'\x68(....)\x68(....)\x6a\x00\x68(....)'

    for m in re.finditer(key_dw_len_egg, file_data):
         keys.append( {'key':m.group(2) + m.group(1), 'length':struct.unpack('<I',m.group(3))[0]})      

    out_strings = []
    
    for sj in stack_strings:
        s = sj.get('data')
        str_len = len(s)
        flag_found = False
        tmp_out = []
        for k in keys:
            if k.get('length') == str_len:
                out = xor_decrypt(s, k.get('key'))
                if is_ascii(out):
                    tmp_out.append(out)
        
        if len(tmp_out) == 0:
            out = xor_decrypt(s[1:-1], bytes([s[0]]))
            if is_ascii(out):
                out_strings.append(out)
        elif len(tmp_out) == 1:
            out_strings.append(tmp_out[0])
        else:
            # This is a hack
            # We have strings that will give valid ascii for multiple keys
            # So far these have only been dll names
            for out in tmp_out:
                if out[:-3].lower() == b'dll':
                    out_strings.append(out)
                    break
    return list(set(out_strings))
            



#### Test All Samples

In [98]:
get_strings(FILE_PATH)

import os

for filename in os.listdir('/tmp/samples'):
    if filename.endswith(".bin"): 
        file_path = os.path.join('/tmp/samples', filename)
        print(f"\n{file_path}")
        print(get_strings(file_path))
        


/tmp/samples/55d329a13da236bec15c4c67686b809a2fbf5f6c9625b62d900ac22a7b729ba9.bin
[b'VirtualAlloc', b'Windows-Update-Agent/11.0.10011.16384 Client-Protocol/2.0\x00', b'rundll32.exe\x00', b'9npSEGB3kg9suo3Yit\x00', b'kernel32.dll', b'https://azure-dbupdate.at/vuUwUx/FyNRoM/index.php\x00', b'Shlwapi.dll\x00', b'WS2_32.dll\x00', b'DllRegisterServer\x00', b'loaddll32.exe\x00', b'Shell32.dll\x00', b'Wininet.dll\x00', b'USER32.dll\x00', b'GetProcAddress', b'LoadLibraryA', b'IPHLPAPI.DLL\x00', b'https://azure-dbupdate.at/vuUwUx/FyNRoM/auth.php\x00', b'https://azure-updatedb.at/vuUwUx/FyNRoM/index.php\x00', b'VirtualFree', b'regsvr32.exe\x00', b'%PROCESSOR_LEVEL%']

/tmp/samples/4b87f95c4477fc66c58b8e16a74f9c47217913cb4a78dc69f27a364a099acd90.bin
[b'VirtualAlloc', b'Windows-Update-Agent/11.0.10011.16384 Client-Protocol/2.0\x00', b'rundll32.exe\x00', b'9npSEGB3kg9suo3Yit\x00', b'kernel32.dll', b'https://azure-dbupdate.at/vuUwUx/FyNRoM/index.php\x00', b'Shlwapi.dll\x00', b'WS2_32.dll\x00', b'Dl

### Sample Using ADV String Obfuscation

A newer sample uses some type of ADV string obfuscation `b9b399dbb5d901c16d97b7c30cc182736cd83a7c53313194a1798d61f9c7501e`. We can probably use our old dumpulator tricks for  this.

The start of the `.text` section contains a vtable with all of the string decryption functions.



In [118]:
from dumpulator import Dumpulator, syscall
from dumpulator.native import *

@syscall
def ZwQueryVolumeInformationFile(dp: Dumpulator,
                                 FileHandle: HANDLE,
                                 IoStatusBlock: P(IO_STATUS_BLOCK),
                                 FsInformation: PVOID,
                                 Length: ULONG,
                                 FsInformationClass: FSINFOCLASS
                                 ):
    return STATUS_SUCCESS



dp = Dumpulator("/tmp/b9b.dmp", quiet=True)

dp.start(dp.regs.eip, end=dp.read_ptr(dp.regs.esp))

functs = [0x74040976,0x740409A4,0x740409BA,0x74040998,0x7404098C,0x7403F910,0x7403F91C,0x7403F967,0x7403F9B2,0x7403F9FD,0x7403FA48,0x7403FA93,0x7403FADE,0x7403FB44,0x7403FBAE,0x7403FC20,0x7403FC98,0x7403FD06,0x7403FD74,0x7403FDDD,0x7403FE43,0x7403FEBF,0x7403FF46,0x7403FFD5,0x74040065,0x740400DC,0x74040146,0x740401A2,0x740401FE,0x7404025A,0x740402B6,0x74040312,0x74040378,0x740403D4,0x74040438,0x7404048B,0x740404DE,0x74040542,0x7404059E,0x740405F1,0x74040644,0x740406A0,0x74040706,0x7404074F,0x740407AB,0x74040807,0x7404085A,0x740408B6,0x7404091A]

for fn in functs:
    dp.call(fn,[])


str_tbl_start = 0x7407FDB4
str_tbl_end = 0x7407FE7C

str_tbl_start = 0x7407E000
str_tbl_end = 0x74080224

for ptr in range(str_tbl_start,str_tbl_end,4):
    try:
        ss = dp.read_str(dp.read_ptr(ptr))

        if len(ss) > 4:
            print(ss)
    except:
        continue



Failed to read module data
C:\Users\IEUser\Desktop\DLLLoader32_82D6.exe
"C:\Users\IEUser\Desktop\DLLLoader32_82D6.exe"
e03ed
Uninstall
3fe11
Running exe
Starting the exe with parameters
Run CMD in memory
Run PS in memory
Running dll in memory #3 (DllInstall(Unstall))
Running dll in memory #3 (DllInstall(Install))
Regsvr32 & Execute
MemLoadDllMain || MemLoadExe
zNETjp
5deb9c
Run EXE with admin rights
TAMfm
RunDll32 & Execute
Crypt update & Bots upgrade
Running dll in memory #2 (DllRegisterServer)
tbesqn


#### Different Sample From Same Family

This is clearly a different sample based on the decrypted strings, but it seems to be part of the matanbuchus family... maybe this is a payload instead of a loader? The sample matches analysis from this blog: [Introduction of a PE file extractor for various situations](https://r136a1.info/2022/05/25/introduction-of-a-pe-file-extractor-for-various-situations/)>

We need to figure out why these samples are so different... 

